# Location Recommendation System for New Businesses in Budapest, Hungary &ndash; An app to play with

In this notebook I minimized the number of cells and let you try the recommendation service with different parameters.

First we import some libraries and set up some variables.

In [1]:
!pip install geopy
import requests
import json
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import numpy as np
from sklearn.cluster import MeanShift
import matplotlib.cm as cm
import matplotlib.colors as colors
from sys import exit
CLIENT_ID = 'ZBC2EWETE1R2DGRBAWJ1YGOOUBCTXCOZTJS4JHQ0V1MBJPJU' # your Foursquare ID
CLIENT_SECRET = 'UGTY5QSD0CVQQC1DFIKRZULRMNVZJSFAPKVHWKSZXR0ADP5Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
intent = "browse"
LIMIT = 500
BP_LAT = 47.495
BP_LNG = 19.05

url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION)

results = requests.get(url).json()
major_category_list = results['response']['categories']
df_categories = pd.DataFrame()

for major_category in major_category_list:
    for minor_category in major_category['categories']:
        df_categories = df_categories.append({
            "name": minor_category['name'],
            "id": minor_category['id']
        }, ignore_index=True)

## Feel free to play with these parameters and check out the results below
- **Category:** Enter any category from the [Foursquare Category List](https://developer.foursquare.com/docs/build-with-foursquare/categories/). Please, make sure to enter them exactly as they are listed. A few examples that may worth to try: Restaurant, Coffee Shop, Bookstore, Jewelry Store, Used Bookstore, etc.
- **Number of recommendations:** The number of recommendations you would like to have.
- **BANDWIDTH:** This is a necessary parameter for the Mean Shift algorithm. The smaller the bandwidth, the denser and smaller the clusters that are identified. 0.002 is a reasonable value for most venue categories.

In [30]:
category = "Tea Room"
number_of_reccommendations = 5
BANDWIDTH = 0.002

### Now, if you execute the cell below, you should see the recommended locations of your new business on the map
It may take a couple of seconds until it executes, so please be patinet.

In [31]:
lat, lng = np.mgrid[47.475:47.516:0.01, 19.02:19.081:0.01]
ll = np.mgrid[47.475:47.516:0.01, 19.02:19.081:0.01].reshape(2,-1).T

try:
    categoryId = df_categories[df_categories['name'] == category].iloc[0,0]
except:
    print("Sorry, there is not such a category")


url_list = []
for i in range(lat.shape[0] - 1):
    for j in range(lat.shape[1] - 1):
        sw_lat = lat[i][j]
        sw_lng = lng[i][j]
        ne_lat = lat[i+1][j+1]
        ne_lng = lng[i+1][j+1]
        sw = "{:.3f},{:.3f}".format(sw_lat, sw_lng)
        ne = "{:.3f},{:.3f}".format(ne_lat, ne_lng)

        url_list.append('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&limit={}&intent={}&sw={}&ne={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET,
            VERSION,
            LIMIT,
            intent,
            sw,
            ne,
            categoryId))
        
coordlist = []
for url in url_list:
    results = requests.get(url).json()
    venues = results['response']['venues']
    for venue in venues:
        lat = venue['location']['lat']
        lng = venue['location']['lng']
        coordlist.append([lat, lng])
        
num_results = len(coordlist)
print("We have found {} {}s.".format(len(coordlist), category))

X = np.array(coordlist)
clust = MeanShift(BANDWIDTH).fit(X)
unique, counts = np.unique(clust.labels_, return_counts=True)
good_clusters = counts.argsort()[-number_of_reccommendations:][::-1]
X_good = X[[label in good_clusters for label in clust.labels_]]
X_good_labels = MeanShift(BANDWIDTH).fit(X_good).labels_

print("Please see the lcoation recommendation in the map below.")
print("Black markers are the cluster centers, colored ones are the already existing \nvenues from the category in question")
map = folium.Map(location=[BP_LAT, BP_LNG], zoom_start=13)

x = np.arange(number_of_reccommendations+1)
ys = [i + x + (i*x)**2 for i in range(number_of_reccommendations+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

for coord, cluster in zip(X_good, X_good_labels):
    folium.CircleMarker(
        coord,
        radius=5,
        #color="blue",
        color=rainbow[cluster+1],
        fill=True,
        fill_color=rainbow[cluster+1],
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
for coord in clust.cluster_centers_[good_clusters]:
    folium.CircleMarker(
        coord,
        radius=4,
        color='#0d0d0c',
        fill=True,
        fill_color='#52524e',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
    
map

We have found 62 Tea Rooms.
Please see the lcoation recommendation in the map below.
Black markers are the cluster centers, colored ones are the already existing 
venues from the category in question
